# How To Use NLP

## 2. Word2Vec으로 문장을 벡터로 변환하기

$ pip install gensim

### 2.1 Genism으로 토지 읽어들이고 Word2Vec로 embedding하기

In [1]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Okt

#### utp-16인코딩으로 파일을 열고 글자를 출력하기

In [2]:
fp = codecs.open('../data/BEXX0003.txt','r', encoding='utf-16')
soup = BeautifulSoup(fp, "html.parser")
body = soup.select_one("body > text")
text = body.getText()

#### 텍스트를 한 줄씩 처리하기

In [3]:
okt = Okt()

In [4]:
results = []
#split by lines
lines = text.split("\r\n")

for line in lines:
    # 형태소 분석하기 - 단어의 기본형 사용
    poslist = okt.pos(line, norm=True, stem=True)
    r = []
    for word in poslist:
        # 어미/조사/구두점 등은 대상에서 제외 
        if not word[1] in ["Josa", "Eomi", "Punctuation"]:
            r.append(word[0])
    readlines = (" ".join(r)).strip()
    results.append(readlines)
    print(readlines[:50])
    print(len(readlines))


제 1 편 어둠 발 소리 
 서다 序 
 1897년 의 한가위 
 까치 들 울타리 안 감나
173191


#### 파일로 출력하기

In [5]:
origins = '../data/toji.origin'
with open(origins, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

#### Word2Vec 모델 만들기

word2vec 파라메터 설명
config = {  
    'min_count': 5,  # 등장 횟수가 5 이하인 단어는 무시  
    'window' : skip gram에서, 주변에 몇개의 단어까지 볼 것인가. 보통 5단어 정도로 한다  
    'hs' : hirachical softmax, negative sampling flag, 모델 복잡도를 개선하기 위해 사용함, 보통 negative sampling의 성능이 더 좋음  
    'size': 300,  # 300차원짜리 벡터스페이스에 embedding, 보통 300차원 정도로 한다.  
    'sg': 1,  # 0이면 CBOW, 1이면 skip-gram을 사용한다. 보통 skip-gram을 많이 사용  
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수  
    'iter': 5,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수  
    'workers': multiprocessing.cpu_count(),  
}  

In [7]:
?word2vec.Word2Vec

Init signature:
word2vec.Word2Vec(
    sentences=None,
    corpus_file=None,
    vector_size=100,
    alpha=0.025,
    window=5,
    min_count=5,
    max_vocab_size=None,
    sample=0.001,
    seed=1,
    workers=3,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    negative=5,
    ns_exponent=0.75,
    cbow_mean=1,
    hashfxn=<built-in function hash>,
    epochs=5,
    null_word=0,
    trim_rule=None,
    sorted_vocab=1,
    batch_words=10000,
    compute_loss=False,
    callbacks=(),
    comment=None,
    max_final_vocab=None,
    shrink_windows=True,
)
Docstring:     
Serialize/deserialize objects from disk, by equipping them with the `save()` / `load()` methods.

Warnings
--------
This uses pickle internally (among other techniques), so objects must not contain unpicklable attributes
such as lambda functions etc.
Init docstring:
Train, use and evaluate neural networks described in https://code.google.com/p/word2vec/.

Once you're finished training a model (=no more updates, only queryi

In [8]:
from gensim.models import word2vec

data = word2vec.LineSentence(origins)
model = word2vec.Word2Vec(data, vector_size=200, window=10, hs=1, min_count=2, sg=1)
model.save("../data/toji.bin")
print("ok")

ok


### 2.2 사용한 language model을 바탕으로 유사단어 검출하기

In [9]:
model = word2vec.Word2Vec.load("../data/toji.bin")

In [10]:
model.wv.most_similar(positive =["땅"])

[('조상', 0.8641260266304016),
 ('치우다', 0.8482598066329956),
 ('꾼', 0.8458912372589111),
 ('누님', 0.8402015566825867),
 ('사대부', 0.8396265506744385),
 ('은덕', 0.8389222025871277),
 ('작정', 0.8363561630249023),
 ('보름', 0.833772599697113),
 ('대가', 0.832798957824707),
 ('골골', 0.8297581076622009)]

In [11]:
model.wv.most_similar(positive=["땅","어머니"])

[('신도비', 0.9401202201843262),
 ('천하다', 0.9393921494483948),
 ('우천', 0.9373031258583069),
 ('혈육', 0.9363511800765991),
 ('만석꾼', 0.9271273016929626),
 ('만민', 0.9253895878791809),
 ('어무님', 0.9241967797279358),
 ('드나들다', 0.9238700270652771),
 ('일체', 0.9226057529449463),
 ('이쯤', 0.9224104881286621)]

## Summerize : Word2Vec을 이용하여 특정 단어의 유의어, 반의어 등 추출 가능  
**Word2Vec으로 단어를 linear expression이 가능하다.**